In [2]:
### Imports for the Data Preprocessing
import numpy as np
import pandas as pd
import matplotlib as plt
import os
import gc
from typing import Tuple
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
from imblearn.over_sampling import SMOTE
import numpy as np


In [5]:
# get working directory and remove last folder
# TODO: make this more robust
wd = os.path.dirname(os.getcwd())
os.chdir(wd)
print('Working Directory: ', os.getcwd())

Working Directory:  n:\MASTER_DS\Code\Kaggle_competition\Kaggle-seminar\student-performance


In [3]:
# one cell to load the train and label data
def load_train_data(file_path: str , dtypes: dict = None, n_rows: int = None):
    # If dtypes is not specified, set default data types for each column
    if dtypes is None:
        dtypes = {
            'level': np.uint8,  
            'session_id': np.int64,
            'level_group': 'category',
            'event_name': np.uint8,
            'name': np.uint8,
            'fqid': np.uint8,
            'room_fqid': np.uint8,
            'text_fqid': np.uint8,
            'fullscreen': np.uint8,
            'hq': np.uint8,
            'music': np.uint8,
            'hover_duration_mean': np.float32,
            'difference_clicks_mean': np.float32,
            "distance_clicks_mean": np.float32,
            "screen_distance_clicks_mean": np.float32,            
            'elapsed_time_std': np.float32,
            'page_std': np.float32,
            'room_coor_x_std': np.float32,
            'room_coor_y_std': np.float32,
            'screen_coor_x_std': np.float32,
            'screen_coor_y_std': np.float32,
            'hover_duration_std': np.float32,
            'difference_clicks_std': np.float32,
            "distance_clicks_std": np.float32,
            "screen_distance_clicks_std": np.float32,
            'index_sum_of_actions': np.int32,
            'difference_clicks_max': np.float32,
            'elapsed_time_max': np.float32,
            'clicks_per_second': np.float32,
            "sum_distance_clicks_max": np.float32,
        }
        
    # Read in the CSV file
    if n_rows is None:
        df = pd.read_csv(file_path, dtype=dtypes, index_col = 0)
    else:
        df = pd.read_csv(file_path, dtype=dtypes, nrows=n_rows, index_col= 0)
    
    # Set data types for columns with "_i" index in their name
    row, cols = df.shape
    if cols > 50:
        for column in df.columns:
            base_name = column.rsplit('_', 1)[0]  # get the base name by splitting on the last "_" character
            if base_name in dtypes:
                column_number = column.rsplit('_', 1)[1]  # get the number from the index by splitting on the last "_" character
                new_column_name = f"{base_name}_{column_number}"  # construct the new column name
                column_dtype = dtypes[base_name]
                df[new_column_name] = df[column].astype(column_dtype)  # set the same data type for all columns with the same base name

    return df



In [6]:
train_data = load_train_data("data/processed/df_5_12_flattened.csv")
dtypes_labels= {
    'correct': np.uint8, 
    'q':np.uint8}

labels = pd.read_csv('data/processed/labels_q4-13.csv', dtype=dtypes_labels)

In [7]:
merged_df = pd.merge(train_data, labels, left_on='session_id_1',
                     right_on='session', how='inner')
merged_df.drop(['session_id_1', 'session'], axis=1, inplace=True)
grouped_df = merged_df.groupby('q')



In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
import numpy as np
# Dictionary to store the models
models = {}

# Hyperparameters
hidden_units = [512, 256, 128]
learning_rate = 0.001
epochs = 20
batch_size = 32



from sklearn.utils import class_weight

# Calculate class weights
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)

for question, group in grouped_df:
    # Access the features and labels for the current question
    X = group.drop(['q', 'correct', 'level_group_1'], axis=1)
    y = group['correct']

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Calculate class weights for the current question
    class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)

    # Define the MLP architecture
    model = Sequential()
    model.add(Dense(hidden_units[0], activation='relu', input_shape=(X.shape[1],)))
    model.add(Dense(hidden_units[1], activation='relu'))
    model.add(Dense(hidden_units[2], activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model with class weights
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model with class weights
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, class_weight=class_weights, verbose=0)

    # Store the model in the dictionary
    models[question] = model

    # Evaluate the model on the testing set
    y_pred = model.predict(X_test)
    y_pred = (y_pred > 0.5)  # Convert probabilities to binary predictions

    # Calculate evaluation metrics
    f1 = f1_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)

    # Print evaluation metrics
    print('Question:', question)
    print('F1 Score:', f1)
    print('Precision:', precision)
    print('Recall:', recall)
    print('Confusion Matrix:')
    print(conf_matrix)

TypeError: compute_class_weight() takes 1 positional argument but 3 were given